## Vine Review Analysis

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
#spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

In [3]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [4]:
# Start a SparkSession
import findspark
findspark.init()

In [5]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-17 23:16:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-10-17 23:16:10 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [10]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

### Create DataFrames to match tables

In [11]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [13]:
# Create the customers_table DataFrame
from pyspark.sql.functions import count
customers_df = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(truncate=False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|15634680   |1             |
|44696507   |1             |
|19901367   |1             |
|11337682   |1             |
|52021773   |1             |
|29916198   |1             |
|42030944   |1             |
|52765209   |1             |
|13519509   |2             |
|39320350   |5             |
|23249582   |1             |
|46351928   |3             |
|44179620   |1             |
|637252     |1             |
|34038793   |1             |
|51136490   |1             |
|49354815   |1             |
|149761     |2             |
|43333798   |1             |
|4657027    |1             |
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00TQ71BXI|Moonlight Mysteri...|
|B00CHZG1FE|Image Line FL Stu...|
|B002JB3BC2|Anti-Virus & Anti...|
|B0113MZ1AC|Windows 8 64 Bit ...|
|B00003069T|ClueFinders 5th G...|
|B008EQUD4U|   Camtasia Studio 8|
|B008IVNFDM|Learn Adobe InDes...|
|B001Q24UAM|Play! The Ultimat...|
|B005HMUUSI|Mariner StoryMill...|
|B00005MG8G|My Deluxe MailLis...|
|B003L51CZ8|Sony Vegas Movie ...|
|B00JFYWY2Y|Learning SQL - Tr...|
|B00DDUDTZ2|Anno 2070 Complet...|
|B004625PV2|My Golf Game feat...|
|B0002TXT7K|Epic Pinball: The...|
|B001J8KF4I|Early Learning Fu...|
|B000KUWQ98|Extreme Behavior ...|
|B00O2QS9GU|Access 2013 Train...|
|B00NNU0C3Y|MAGIX Music Maker...|
|B00SLV70HQ|Can-Am Commander ...|
+----------+--------------------+
only showing top 20 rows



In [23]:
# Create the review_id_table DataFrame.
review_id_table = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_table = review_id_table.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|   42605767|B00MUTIDKI|     248732228| 2015-08-31|
|R12NR0R5A9F7FT|   51771800|B00EPACNUG|     531462352| 2015-08-31|
|R1LSH74R9XAP59|   16053526|B00164AZA4|     473982505| 2015-08-31|
|R1QXUNTF76K7L6|   15319481|B00E6LIEFM|     189774198| 2015-08-31|
|R2F7DR75PS8NKT|    1441820|B00VWEBG06|     852470365| 2015-08-31|
|R2C1DJSCC8UFS6|   37107850|B00EP7AP7C|     279360628| 2015-08-31|
|R1AXGS1W4YFXMX|     302120|B00OW2PET4|     729971168| 2015-08-31|
|R1XU1B93402SYJ|   20193077|B00N4OLCRO|     776572654| 2015-08-31|
|R2U432NB3OPVR0|   13106017|B005CELN8W|     222071424| 2015-08-31|
|R3R6FIMIOQ5SP9|   32587108|B005CELL1G|     168801430| 2015-08-31|
|R12TX6V09C9QNQ|   32020017|B00H09BOXQ|     455502995| 2015-08-31|
|R33UCII6YKUMKV|   13376158|B00MCLGAAO|     652069521| 2015-08

In [16]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [25]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://m16-amazon-challenge.c3oqll57cdog.us-east-2.rds.amazonaws.com:5432/m16challenge"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
# 8 minutes 57 seconds
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# 2 minutes 45 seconds
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 9 minutes 55 seconds
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)